<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/veo2_video_generation_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| | |
|-|-|
|Author(s) | [Frank Morales](https://github.com/frank-morales2020) |

### Install Google Gen AI SDK for Python and other libraries

In [40]:
%pip install --upgrade --quiet google-genai
%pip install -q mediapy

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [4]:
import time
import urllib

from PIL import Image as PIL_Image
from google import genai
from google.genai import types
import matplotlib.pyplot as plt
import mediapy as media

In [ ]:
!pip install colab-env -q
import colab_env
colab_env.RELOAD()

In [ ]:
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
print(f"Project ID: {PROJECT_ID}")

In [11]:
import os

# Get project details from environment variables
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION")
SERVICEACCOUNT = os.environ.get("GOOGLE_CLOUD_SERVICEACCOUNT")
PROJECT_NUMBER = os.environ.get("GOOGLE_CLOUD_PROJECT_NUMBER")
BUCKET_NAME = os.environ.get("GOOGLE_CLOUD_BUCKET_NAME")
STAGING_BUCKET = f"gs://{BUCKET_NAME}/staging"


client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [12]:
def show_video(gcs_uri):
    file_name = gcs_uri.split("/")[-1]
    !gsutil cp {gcs_uri} {file_name}
    media.show_video(media.read_video(file_name), height=500)


def display_images(image) -> None:
    fig, axis = plt.subplots(1, 1, figsize=(12, 6))
    axis.imshow(image)
    axis.set_title("Starting Image")
    axis.axis("off")
    plt.show()

### Load the video generation model

In [13]:
video_model = "veo-2.0-generate-001"

### Generate videos from a text prompt

With Veo 2, you have the option to generate 8 second videos from a text prompt. In order to generate a video in the following sample, specify the following info:
- **Prompt:** A detailed description of the video you would like to see.
- **Aspect ratio:** Select either 16:9 or 9:16.
- **File location:** The generated video will be shown below with support from a previously defined helper function. The video will also be stored in Cloud Storage once video generation is complete. Specify the file path where you would like this video to be stored in the output_gcs field.
- **Number of videos:** Set this value to 1 or 2.
- **Video duration:** Can 5, 6, 7, or 8 seconds.
- **Prompt enhancement:** The `veo-2.0-generate-001` model offers the option to enhance your provided prompt. To utilize this feature, set `enhance_prompt` to True. A new, detailed prompt will be created from your original one to help generate higher quality videos that better adhere to your prompt's intent.

In [17]:
from google.oauth2 import service_account

# Change the file path if it's different in your environment
credentials = service_account.Credentials.from_service_account_file(
    '/content/gdrive/MyDrive/data/gen-lang-client-0870511801-3d0598c7b654.json',
    scopes=['https://www.googleapis.com/auth/cloud-platform'] # Add scopes here
)

client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION,
    credentials=credentials, # Passing credentials here
)

In [ ]:
# Your detailed prompt
prompt = """Cinematic, futuristic visualization: An advanced AI agent is actively designing a complex flight plan. Show a sleek, semi-transparent holographic interface displaying a rotating Earth globe with dynamic flight paths being calculated in real-time. Visualize abstract representations of foundational concepts weaving into the AI's process:
1.  Newtonian physics represented by elegant, arcing trajectory lines influenced by gravity and thrust vectors.
The overall mood should be one of sophisticated intelligence, high technology, and controlled complexity. Focus on the interaction of data, algorithms, and the evolving flight path on the holographic display. High detail, smooth animation, dramatic lighting."""


aspect_ratio = "16:9"  # ["16:9", "9:16"]


# Use STAGING_BUCKET for output
output_gcs = STAGING_BUCKET

operation = client.models.generate_videos(
    model=video_model,
    prompt=prompt,
    config=types.GenerateVideosConfig(
        aspect_ratio=aspect_ratio,
        output_gcs_uri=output_gcs,
        number_of_videos=1,
        duration_seconds=5,
        person_generation="dont_allow",
        enhance_prompt=True,
    ),
)

while not operation.done:
    time.sleep(15)
    operation = client.operations.get(operation)
    print(operation)

#if operation.response:
#    show_video(operation.result.generated_videos[0].video.uri)

if operation.response and operation.result and operation.result.generated_videos:
    show_video(operation.result.generated_videos[0].video.uri)
else:
    print(f"Video generation failed or did not return the expected results.\nOperation details: {operation}")


When generating videos of people you can also set the `person_generation` parameter accordingly:
* `person_generation`: allow_adult, dont_allow

In [ ]:

# Your detailed prompt
prompt = """Cinematic, futuristic visualization: An advanced AI agent is actively designing a complex flight plan. Show a sleek, semi-transparent holographic interface displaying a rotating Earth globe with dynamic flight paths being calculated in real-time. Visualize abstract representations of foundational concepts weaving into the AI's process:
2.  Galilean observation symbolized by streams of real-time data (weather patterns, sensor readings) flowing into the system and being analyzed.
3.  Einsteinian relativity subtly hinted at through visualizations of optimized path calculations accounting for complex variables, perhaps shown as warping grids or spacetime ripples affecting the flight path optimization.
4.  Hinton's neural networks visualized as glowing, interconnected nodes learning and adapting the plan based on the incoming data streams.
The overall mood should be one of sophisticated intelligence, high technology, and controlled complexity. Focus on the interaction of data, algorithms, and the evolving flight path on the holographic display. High detail, smooth animation, dramatic lighting."""

aspect_ratio = "9:16"  # ["16:9", "9:16"]

operation = client.models.generate_videos(
    model=video_model,
    prompt=prompt,
    config=types.GenerateVideosConfig(
        aspect_ratio=aspect_ratio,
        output_gcs_uri=output_gcs,
        number_of_videos=1,
        duration_seconds=7,
        person_generation="allow_adult",
        enhance_prompt=True,
    ),
)

while not operation.done:
    time.sleep(15)
    operation = client.operations.get(operation)
    print(operation)

if operation.response:
    show_video(operation.result.generated_videos[0].video.uri)

### Generate videos from an image


In [31]:
nature="/content/gdrive/MyDrive/datasets/Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
print()


# Nature
prompt =  "What’s in this image?"
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

!gsutil cp {nature} {STAGING_BUCKET}/nature.jpg

print()
print('The image url is here: %s'%image_url)


Copying file:///content/gdrive/MyDrive/datasets/Gfp-wisconsin-madison-the-nature-boardwalk.jpg [Content-Type=image/jpeg]...
-
Operation completed over 1 objects/1.0 MiB.                                      

The image url is here: https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg


In [ ]:
image_show = PIL_Image.open(
    urllib.request.urlopen(
        "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/flowers.png"
    )
)
display_images(image_show)

import io

# Save the image to a BytesIO object
image_bytes = io.BytesIO()
image_show.save(image_bytes, format='PNG')  # Adjust format if needed

# Upload to your bucket
with open('/tmp/image_to_upload.png', 'wb') as f:
    f.write(image_bytes.getvalue())
!gsutil cp /tmp/image_to_upload.png {STAGING_BUCKET}/image_show.png

In [38]:
!gsutil ls {STAGING_BUCKET}/image_show.png

gs://poc-my-new-staging-bucket-2025-1/staging/image_show.png


In [ ]:
image_show = PIL_Image.open(
    urllib.request.urlopen(
        "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
    )
)
display_images(image_show)

In [ ]:
image_gcs = (
    "gs://poc-my-new-staging-bucket-2025-1/staging/image_show.png"  # {type: 'string'}
)
aspect_ratio = "16:9"  # ["16:9", "9:16"]

operation = client.models.generate_videos(
    model=video_model,
    image=types.Image(
        gcs_uri=image_gcs,
        mime_type="image/png",
    ),
    config=types.GenerateVideosConfig(
        aspect_ratio=aspect_ratio,
        output_gcs_uri=output_gcs,
        number_of_videos=1,
        duration_seconds=8,
        person_generation="allow_adult",
    ),
)

while not operation.done:
    time.sleep(15)
    operation = client.operations.get(operation)
    print(operation)

if operation.response:
    show_video(operation.result.generated_videos[0].video.uri)